# Import libraries

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from gurobipy import *

# Read and preprocess data

In [12]:
import csv
from datetime import datetime
import os

def load_data(file_path):
    data_dict = dict()

    with open(file_path) as file:
        reader = csv.DictReader(file)
        for row in reader:
            if row['Numéro du groupe'] != '':
                heures, minutes, _ = map(int, row['TransitTime'].split(':'))
                total_minutes = heures * 60 + minutes
                data_dict[int(float(row['Numéro du groupe']))] = (
                    int(float(row['Femmes'])),
                    int(float(row['Hommes'])),
                    int(float(row['WCHR'])),
                    total_minutes
                )

    return data_dict

# Specify the list of files
file_names = ["5Nov.csv", "7Nov.csv", "22Oct.csv", "24Oct.csv", "23Oct.csv", "30Oct.csv", "DataSeating 2024.csv"]

# Define the 'data' directory
data_directory = 'data'

# Load data for each file
Nov5, Nov7, Oct22, Oct24, Oct23, Oct30, Date0 = [load_data(os.path.join(data_directory, file)) for file in file_names]

In [13]:
Passengers = {}
j = 1

for group_num, data in Nov5.items():
    weights = {'Femmes': 70, 'Hommes': 85, 'WCHR': 100}  # Weight per passenger
    
    # Extracting data
    femmes, hommes, wchr, total_minutes = data
    
    # Enumerating passengers
    for _ in range(femmes + hommes + wchr):
        if femmes > 0:
            weight = weights['Femmes']  
            Passengers[f'Passenger_{j}'] = {'gender': 'Femmes', 'group': group_num, 'weight': weight, 'connection_time': total_minutes}
            femmes -= 1
        elif hommes > 0:
            weight = weights['Hommes']
            Passengers[f'Passenger_{j}'] = {'gender': 'Hommes', 'group': group_num, 'weight': weight, 'connection_time': total_minutes}
            hommes -= 1
        else:
            weight = weights['WCHR']
            Passengers[f'Passenger_{j}'] = {'gender': 'WCHR', 'group': group_num, 'weight': weight, 'connection_time': total_minutes}
            wchr -= 1
        j += 1

print(Passengers)

{'Passenger_1': {'gender': 'Femmes', 'group': 1, 'weight': 70, 'connection_time': 0}, 'Passenger_2': {'gender': 'Hommes', 'group': 1, 'weight': 85, 'connection_time': 0}, 'Passenger_3': {'gender': 'Hommes', 'group': 1, 'weight': 85, 'connection_time': 0}, 'Passenger_4': {'gender': 'Hommes', 'group': 2, 'weight': 85, 'connection_time': 60}, 'Passenger_5': {'gender': 'Femmes', 'group': 3, 'weight': 70, 'connection_time': 165}, 'Passenger_6': {'gender': 'Hommes', 'group': 3, 'weight': 85, 'connection_time': 165}, 'Passenger_7': {'gender': 'Femmes', 'group': 4, 'weight': 70, 'connection_time': 0}, 'Passenger_8': {'gender': 'Hommes', 'group': 4, 'weight': 85, 'connection_time': 0}, 'Passenger_9': {'gender': 'WCHR', 'group': 4, 'weight': 100, 'connection_time': 0}, 'Passenger_10': {'gender': 'Femmes', 'group': 5, 'weight': 70, 'connection_time': 0}, 'Passenger_11': {'gender': 'Femmes', 'group': 5, 'weight': 70, 'connection_time': 0}, 'Passenger_12': {'gender': 'Femmes', 'group': 5, 'weight':

In [11]:
Passengers['Passenger_1']['weight']

{'gender': 'Femmes', 'group': 1, 'weight': 70, 'connection_time': 0}

# Variables

In [2]:
ranks = 29 #nombre de rangées dans l'avion
n = 81 #nombre de passagers

#Passengers : dictionnaire extrait du fichier excel
#Les sièges sont comptés comme une liste et pas comme une matrice

# -- Initialisation du modèle --
# m : Model
    
m=Model('statique')

# -- Ajout des variables  --
# S: dict[(int : int) : Var]  
# Sij : var. binaire qui à un passager i associe le siège j
S = {(i,j) : m.addVar(vtype = GRB.BINARY, name = f'j{i}') for i in range(1, n+1) for j in range (1, 6*ranks+1)}

''' # complete after
def modele_statique():
    m=Model('statique')
    #Sij : var. binaire qui à un passager i associe le siège j
    S = {(i,j) : m.addVar(vtype = GRB.BINARY, name = f'j{i}') for i in range(1, n+1) for j in range (1, 6*ranks+1)}

    return m
'''

# Constraints

**Un passager maximum par siège:** 

$$\sum_{i=1}^{np} S_{ij} \leq 1 ~~\forall j $$

Un siège maximum par passager

$$\sum_{j=1}^{ns} S_{ij} = 1 ~~\forall i $$

Barycentre dans la zone de centrage.

$$ x = \sum_{i=1}^{np}   \sum_{j=1}^{ns} \dfrac{(j\%6)S_{i,j}w_i)}{\sum_{i=1}^{np} w_i}$$

In [ ]:
x = quicksum(((i%6)*[S[(i, j)]*Passengers_weight[(i,j)] for i in range(1, np + 1) for j in range(1, ns + 1)])(quicksum([Passengers_weight[(i, j)] for i in range(1, np + 1)])))